In [1]:
from gensim import corpora
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
from string import digits
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import xgboost as xgb
import numpy as np
import sklearn as sk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble, metrics, model_selection, naive_bayes, pipeline
from sklearn.decomposition import NMF, LatentDirichletAllocation
from stop_words import get_stop_words
import gensim
import re
from gensim.models import ldamodel as ld

c:\users\radames\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [3]:
# нулей не должно быть!


def sum_up_word2vec_array(clnd_text):
    
    total = None
    
    for word in clnd_text:
        if word in w2v:
            if total is None:
                total = w2v[word]
            else:
                total = np.add(total, w2v[word])
                
    if total is None:
        return np.zeros(w2v_array_len)
    else:
        return total
    

In [4]:
# нулей не должно быть!


def avg_up_word2vec_array(clnd_text):
    
    total = None
    
    for word in clnd_text:
        if word in w2v:
            if total is None:
                total = w2v[word]
            else:                
                total = np.mean([total, w2v[word]], axis=0)
                
                
    if total is None:
        return np.zeros(w2v_array_len)
    else:
        return total
    

In [5]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [6]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)
eng_stopwords = set(stopwords.words('english')).union(set(get_stop_words('english')))

In [7]:
def lexical_diversity(file_text):
    file_text = file_text.translate(remove_digits)
    try:
        tokens = nltk.word_tokenize(file_text)
    except:
        nltk.download('punkt')
        tokens = nltk.word_tokenize(file_text)
        

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return len(set(tokens))/len(tokens)

In [8]:
# вытаскиваем "значимые" слова




raw_documents_authors = ['', '', '']


for index, row in df_train.iterrows():
    
    if row['author'] == 'EAP':
        raw_documents_authors[0] += row['cleaned_text_string'] + ' '
    elif row['author'] == 'HPL':
        raw_documents_authors[1] += row['cleaned_text_string'] + ' '
    else:
        raw_documents_authors[2] += row['cleaned_text_string'] + ' '
        

# удалим уникальные слова, не встречающиеся у других писателей

eap_only = set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[1].split(' ')) - set(raw_documents_authors[2].split(' '))
hpl_only = set(raw_documents_authors[1].split(' ')) - set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[2].split(' '))
msh_only = set(raw_documents_authors[2].split(' ')) - set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[1].split(' '))

unique_words = eap_only.union(hpl_only).union(msh_only)

tf = TfidfVectorizer(analyzer='word')
idf_matrix =  tf.fit_transform(raw_documents_authors)
feature_names = tf.get_feature_names()
# dictionary_word = dict(zip(feature_names, idf_matrix))

dense_idf = [i.todense() for i in idf_matrix]
print(dense_idf)

max_weighted_term = []

eap_dense_list = dense_idf[0].tolist()[0]
hpl_dense_list = dense_idf[1].tolist()[0]
mws_dense_list = dense_idf[2].tolist()[0]

for inum, i in enumerate(eap_dense_list):
    max_weighted_term.append(max(hpl_dense_list[inum], mws_dense_list[inum], 
                             i))

max_tf_dict = dict(zip(feature_names, max_weighted_term))



[matrix([[ 0.00053369,  0.00053369,  0.00106738, ...,  0.00053369,
          0.        ,  0.        ]]), matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.00135189,  0.00067595]]), matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])]


In [9]:
model = gensim.models.Word2Vec(df_train['cleaned_text'], size=100)

In [10]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [11]:
w2v_array_len = list(w2v.items())[0][1].shape[0]

In [12]:
def extract_top_words(tfidfdict, numwrd):

    top_word_dict, min_value, min_key = {}, 99, ''
    

    for k, v in max_tf_dict.items():
        # print(top_word_dict.values())
        # print(v)
        if k not in unique_words and k not in eng_stopwords:
        
            if len(top_word_dict) < numwrd:
                top_word_dict[k] = v
                if v <= min_value:
                    min_key = k
            else:
                # print(v, min(list(top_word_dict.values())))
                if v > min(list(top_word_dict.values())) and k not in eng_stopwords:

                    min_value = min(top_word_dict.values())

                    for ky, va in top_word_dict.items():
                        if va == min_value:
                            min_key = ky

                    top_word_dict.pop(min_key)
                    top_word_dict[k] = v
                
    return top_word_dict
another_top_words_dict = extract_top_words(max_tf_dict, 80)
high_tf_idf_words_columns = list(another_top_words_dict.keys())


def count_topwords(target_df):

    for word in high_tf_idf_words_columns:
        
        # TODO: костыль, нужен, когда у нас уже есть такие столбцы
        # в датасете
#         try:
#             target_df = target_df.drop(word, 1)
#         except ValueError:
#             pass
        

        def count_numwords(collist):
            value = 0

            for wd in collist:
                if wd == word:
                    value += 1
            return value


        target_df[word] = target_df.cleaned_text.apply(count_numwords)
        


        

In [13]:
df_train['length']=df_train['cleaned_text_string'].apply(len)
df_train["num_words"] = df_train["text"].apply(lambda x: len(str(x).split()))
df_train["num_unique_words"] = df_train["text"].apply(lambda x: len(set(str(x).split())))
df_train["num_punctuations"] =df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_train["num_words_upper"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_train["num_words_title"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_train["mean_word_len"] = df_train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_train["num_stopwords"] = df_train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_train['lexical_diversity'] = df_train.text.apply(lexical_diversity)
df_train['w2v_array'] = df_train.cleaned_text.apply(sum_up_word2vec_array)
count_topwords(df_train)

df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,...,chang,mani,fear,one,thus,certain,still,return,saw,see
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,...,0,0,0,0,0,0,0,1,0,0
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,...,0,0,0,0,0,0,0,0,0,0
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def create_w2v_columns(target_df):
    
    # сначала вытаскиваем колонку как список списков
    
    w2v_array = target_df['w2v_array'].tolist()
    
    for i in range(100):
        
        target_df['w2v_feature_' + str(i)] = [j[i] for j in w2v_array]

In [15]:
create_w2v_columns(df_train)

In [16]:
df_train.head()

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,...,w2v_feature_90,w2v_feature_91,w2v_feature_92,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,...,-2.663512,-1.303735,-0.877374,-7.937459,-4.798588,-7.241487,-9.307872,-3.895346,3.014773,10.633761
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,...,-1.008078,-0.375688,-0.311576,-2.121509,-1.593851,-1.819666,-2.784693,-1.001561,0.877215,3.173744
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,...,-0.152554,-1.109466,-0.952268,-7.125866,-2.325616,-7.382561,-6.925181,-4.383532,2.261675,8.054424
3,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixteen...,144,34,32,4,0,...,-1.126091,-1.244493,-1.154061,-7.403440,-3.271107,-7.417303,-8.028737,-4.295195,2.497749,9.283039
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon at...,102,27,25,4,0,...,-1.407288,-0.927115,-0.816135,-4.990958,-2.828885,-4.729518,-5.940284,-2.799661,1.966485,6.781491


In [17]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,lexical_diversity,happi,...,w2v_feature_90,w2v_feature_91,w2v_feature_92,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99
count,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,...,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000
mean,81.543165,25.442405,21.894937,4.096329,0.553291,2.102405,4.644952,12.747595,0.886060,0.004684,...,-1.597563,-0.711742,-0.616946,-4.337550,-2.765478,-3.670272,-4.922817,-2.245777,1.494961,6.133319
std,60.100183,18.567706,13.727397,3.573788,0.892966,2.052241,0.631340,9.619779,0.097354,0.070110,...,2.060546,0.492144,0.494714,3.051287,2.432590,3.045421,3.386442,1.728497,1.034629,4.427708
min,5.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.333333,0.000000,...,-43.132179,-7.327130,-10.641767,-45.374699,-54.478886,-32.246254,-54.783573,-19.000381,0.000000,0.000000
25%,40.000000,12.000000,12.000000,2.000000,0.000000,1.000000,4.250000,6.000000,0.821429,0.000000,...,-2.045488,-0.911061,-0.793333,-5.618233,-3.482996,-5.050953,-6.285369,-2.985216,0.788991,3.167969
50%,66.000000,21.000000,19.000000,3.000000,0.000000,1.000000,4.600000,10.000000,0.894737,0.000000,...,-0.998157,-0.594339,-0.502430,-3.578559,-2.088254,-3.033720,-4.106851,-1.822431,1.238525,5.069916
75%,106.000000,33.000000,29.000000,5.000000,1.000000,2.000000,5.000000,17.000000,1.000000,0.000000,...,-0.402371,-0.378131,-0.298529,-2.227310,-1.267485,-1.652519,-2.615695,-1.049155,1.916356,7.795946
max,925.000000,267.000000,155.000000,71.000000,15.000000,43.000000,11.000000,135.000000,1.000000,2.000000,...,2.529351,2.105792,4.669187,0.000000,2.967430,8.583466,0.000000,0.408612,13.935845,91.209198


In [18]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,lexical_diversity,happi,...,w2v_feature_90,w2v_feature_91,w2v_feature_92,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99
count,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,...,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000
mean,86.124586,27.417273,23.544672,3.833719,0.751489,2.124255,4.598182,13.896923,0.883407,0.030609,...,-1.803665,-0.785075,-0.668287,-4.735346,-3.037104,-4.115414,-5.532511,-2.404975,1.703766,6.679250
std,71.976281,23.134440,14.925835,2.840625,1.203636,1.759572,0.561558,12.196599,0.086804,0.180708,...,2.058326,0.667723,0.593003,4.007942,2.736596,3.838224,4.640517,2.154190,1.456653,5.568243
min,4.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.666667,0.000000,0.398990,0.000000,...,-44.951378,-26.515741,-22.135078,-158.751236,-85.285522,-150.461166,-179.890717,-85.041237,0.026838,0.104845
25%,48.000000,15.000000,14.000000,2.000000,0.000000,1.000000,4.250000,7.000000,0.823529,0.000000,...,-2.252982,-0.979568,-0.849959,-5.920853,-3.670478,-5.349403,-6.842672,-3.101918,0.957934,3.778657
50%,74.000000,23.000000,21.000000,3.000000,0.000000,2.000000,4.560791,12.000000,0.885714,0.000000,...,-1.237344,-0.672247,-0.558439,-4.049242,-2.419662,-3.521115,-4.757585,-2.037106,1.459436,5.636068
75%,108.000000,34.000000,30.000000,5.000000,1.000000,3.000000,4.907156,18.000000,0.950000,0.000000,...,-0.615417,-0.445167,-0.350724,-2.676190,-1.562614,-2.122534,-3.140221,-1.265842,2.113236,8.213998
max,2715.000000,861.000000,429.000000,59.000000,27.000000,46.000000,10.500000,437.000000,1.000000,2.000000,...,1.166953,-0.015917,0.125284,-0.069200,-0.032010,3.329655,-0.079661,0.309729,57.268726,208.190536


In [19]:
wordset=set()

In [20]:
#делаю сет со всеми словами
for i in df_train.index:
    wordset |= set(df_train['cleaned_text'][i])
wordlist=list(wordset)

In [21]:
#делаю фрейм со словами
df_word=pd.DataFrame(columns=["word", "mws", "eap", "hpl", "all"])
df_word["word"]=wordlist
df_word["mws"]=0
df_word["eap"]=0
df_word["hpl"]=0
df_word["all"]=0
df_word.head()

,word,mws,eap,hpl,all
0,bland,0,0,0,0
1,ph'nglui,0,0,0,0
2,bacon,0,0,0,0
3,sail,0,0,0,0
4,tudor,0,0,0,0


In [22]:
# как мы будем эту штуку правильнее делать (возможно это жуткий костыль), я хз
# сначала создаем словарь где ключ - уникальное слово, а значение - его порядковый номер
# затем создаем разреженную матрицу, которую заполняем в зависимости от порядковых номеров 
word_dict = {}


In [23]:
#делаю сет со всеми словами
# и сразу заготовку под шапку(потом увидишь зачем)
counter = 0
head = []

for wordlist in df_train['cleaned_text']:
    for word in wordlist:
        if word not in word_dict:
            head.append(word)
            word_dict[word] = counter
            counter += 1


In [24]:
# видоизменять колонки в pandas руками по одному значению в строке или столбце - очень плохая идея
# колонка это numpy.ndarray, а значит при каждой итерации она будет пересоздаваться
# что угробит производительность
# делаем значит так. считаем где сколько и где встречались отдельные слова, затем создаем строку за строкой для 
# каждого предложения

list_of_lists = []

for wordlist in df_train['cleaned_text']:
    row = [0 for i in range(len(word_dict))]
    for word in wordlist:
        row[word_dict[word]] += 1
    list_of_lists.append(row)



In [25]:
print(len(list_of_lists))

19579


In [26]:
# ... и для того чтобы посмотреть встречаемость того или иного слова по авторам добавим такую колонку

count_frame = pd.DataFrame(list_of_lists)
count_frame['author'] = df_train['author']



In [27]:
# ... и теперь нормальную шапку делаем

count_frame.columns = head + ['author']


In [28]:
print(count_frame.head())

   this  process  howev  afford  mean  ascertain  dimens  dungeon  i  might  \
0     1        1      1       1     1          1       1        1  2      1   
1     0        0      0       0     0          0       0        0  0      1   
2     0        0      0       0     0          0       0        0  0      0   
3     0        0      0       0     0          0       0        0  0      0   
4     0        0      0       0     0          0       0        0  0      0   

    ...    aegidus  burr  bentley  waltzer  binder  brusqueri  adriat  ancona  \
0   ...          0     0        0        0       0          0       0       0   
1   ...          0     0        0        0       0          0       0       0   
2   ...          0     0        0        0       0          0       0       0   
3   ...          0     0        0        0       0          0       0       0   
4   ...          0     0        0        0       0          0       0       0   

   agir  author  
0     0     EAP  
1 

In [29]:
# Пока объединим все, потом может быть будем использовать
col=list(count_frame.columns)
col[-1]='author_name'
count_frame.columns=col
pivot_col=pd.pivot_table(count_frame, aggfunc=np.sum, values=col, index=['author_name'])

In [30]:
#Убираем лишние слова, которые не учли раньше
col=list(pivot_col.columns)
col2=[string for string in col if (string[0]!='"' and string[0]!="'"
                                  and string[0]!='.' and string[0]!='`'
                                   and len(string)>3 and '.' not in string)]
col=[]
pivot_col=pivot_col[col2]
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
author_name,,,,,,,,,,,,,,,,,,,,,
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#Create pivot
pivot_col=pivot_col.append(pivot_col.sum(), ignore_index=True)
pivot_col.index=['EAP', 'HPL', 'MWS', 'SUMA']
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
SUMA,1,2,1,48,24,2,2,6,5,2,...,2,1,1,3,1,1,1,1,2,1


In [32]:
summa=[pivot_col.loc['EAP'].sum(), pivot_col.loc['HPL'].sum(), 
       pivot_col.loc['MWS'].sum(), pivot_col.loc['SUMA'].sum()]
pivot_col['summa']=summa
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος,summa
EAP,1,2,0,22,0,2,1,2,3,2,...,0,1,3,1,1,1,1,0,0,87765
HPL,0,0,0,17,24,0,1,3,0,0,...,1,0,0,0,0,0,0,2,1,74269
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,73160
SUMA,1,2,1,48,24,2,2,6,5,2,...,1,1,3,1,1,1,1,2,1,235194


In [33]:
# Create probability of author text knowing that a word was used
pivot_part=pivot_col
pivot_part.loc['EAP']=pivot_col.loc['EAP']/pivot_col.loc['SUMA']
pivot_part.loc['HPL']=pivot_col.loc['HPL']/pivot_col.loc['SUMA']
pivot_part.loc['MWS']=pivot_col.loc['MWS']/pivot_col.loc['SUMA']
pivot_part=pivot_part.loc[['EAP', 'HPL', 'MWS']]
# Delete unique words
pivot_part=pivot_part.loc[:, (pivot_part!=1).all(axis=0)]
pivot_part.head()

,abandon,abash,abat,abbey,abdic,aberr,abhor,abhorr,abil,abject,...,younger,youngest,your,youth,zeal,zenith,zest,zigzag,zone,summa
EAP,0.458333,0.5,0.333333,0.6,0.142857,0.166667,0.058824,0.111111,0.789474,0.333333,...,0.272727,0.2,0.534884,0.101562,0.117647,0.4,0.2,0.4,0.666667,0.373160
HPL,0.354167,0.5,0.500000,0.0,0.000000,0.666667,0.235294,0.555556,0.052632,0.000000,...,0.000000,0.4,0.069767,0.429688,0.470588,0.6,0.2,0.6,0.333333,0.315778
MWS,0.187500,0.0,0.166667,0.4,0.857143,0.166667,0.705882,0.333333,0.157895,0.666667,...,0.727273,0.4,0.395349,0.468750,0.411765,0.0,0.6,0.0,0.000000,0.311062


In [34]:
# It will be easier to work this way
eap_dict=pivot_part.loc['EAP'].to_dict()
hpl_dict=pivot_part.loc['HPL'].to_dict()
mws_dict=pivot_part.loc['MWS'].to_dict()
eap_dict['word']

0.44859813084112149

In [35]:
# Create author score 
def ind_val_eap(listn):
    quant=0
    for word in listn:
        try:
            quant+=eap_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_hpl(listn):
    quant=0
    for word in listn:
        try:
            quant+=hpl_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_mws(listn):
    quant=0
    for word in listn:
        try:
            quant+=mws_dict[word]
        except KeyError:
            quant+=0
    return quant

In [36]:
# Add index of author
df_train['mws_index']=df_train['cleaned_text'].apply(ind_val_mws)/df_train['length']
df_train['eap_index']=df_train['cleaned_text'].apply(ind_val_eap)/df_train['length']
df_train['hpl_index']=df_train['cleaned_text'].apply(ind_val_hpl)/df_train['length']
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,...,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99,mws_index,eap_index,hpl_index
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,...,-7.937459,-4.798588,-7.241487,-9.307872,-3.895346,3.014773,10.633761,0.035935,0.074388,0.034504
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,...,-2.121509,-1.593851,-1.819666,-2.784693,-1.001561,0.877215,3.173744,0.035860,0.060980,0.034739
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,...,-7.125866,-2.325616,-7.382561,-6.925181,-4.383532,2.261675,8.054424,0.026900,0.047832,0.037337


In [37]:
#Transform authors' names to numeric
df_train['author']=df_train['author'].astype('category')
df_train['author2']=df_train['author'].cat.codes
# Create different features 
df_train.head(n=3)
mid = df_train['author2']
df_train.drop(labels=['author2'], axis=1,inplace = True)
df_train.insert(0, 'author2', mid)
df_train.head()

,author2,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,...,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99,mws_index,eap_index,hpl_index
0,0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,...,-7.937459,-4.798588,-7.241487,-9.307872,-3.895346,3.014773,10.633761,0.035935,0.074388,0.034504
1,1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,...,-2.121509,-1.593851,-1.819666,-2.784693,-1.001561,0.877215,3.173744,0.035860,0.060980,0.034739
2,0,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,...,-7.125866,-2.325616,-7.382561,-6.925181,-4.383532,2.261675,8.054424,0.026900,0.047832,0.037337
3,2,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixteen...,144,34,32,4,...,-7.403440,-3.271107,-7.417303,-8.028737,-4.295195,2.497749,9.283039,0.071850,0.033438,0.033601
4,1,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon at...,102,27,25,4,...,-4.990958,-2.828885,-4.729518,-5.940284,-2.799661,1.966485,6.781491,0.036859,0.056661,0.043735


In [38]:
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = df_train['author'].map(author_mapping_dict)
train_id = df_train['id'].values
test_id = df_test['id'].values
cols_to_drop = ['id', 'text']
train_X = df_train.drop(cols_to_drop+['author'], axis=1)
test_X = df_test.drop(cols_to_drop, axis=1)
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [39]:
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [40]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=3, shuffle=True, random_state=194)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

Mean cv score :  0.862729297781


In [41]:
n_comp = 50
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
df_train = pd.concat([df_train, train_svd], axis=1)
df_test = pd.concat([df_test, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [42]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [43]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
df_train["nb_cvec_eap"] = pred_train[:,0]
df_train["nb_cvec_hpl"] = pred_train[:,1]
df_train["nb_cvec_mws"] = pred_train[:,2]
df_test["nb_cvec_eap"] = pred_full_test[:,0]
df_test["nb_cvec_hpl"] = pred_full_test[:,1]
df_test["nb_cvec_mws"] = pred_full_test[:,2]

Mean cv score :  0.450918416166


In [44]:
df_train.head()

,author2,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,...,svd_word_43,svd_word_44,svd_word_45,svd_word_46,svd_word_47,svd_word_48,svd_word_49,nb_cvec_eap,nb_cvec_hpl,nb_cvec_mws
0,0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,...,0.060822,-0.019470,0.022374,0.019055,0.001587,0.037074,0.002873,9.999933e-01,2.752790e-06,3.990111e-06
1,1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,...,0.001035,0.000631,-0.001032,0.003187,-0.004602,0.004110,-0.002629,8.226820e-01,1.492107e-01,2.810727e-02
2,0,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,...,-0.031303,0.011510,-0.014951,-0.028148,0.010121,0.024380,-0.027564,9.999918e-01,8.206128e-06,1.064720e-08
3,2,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixteen...,144,34,32,4,...,0.009119,-0.009910,0.015214,0.013138,0.002048,0.017966,-0.011112,1.436890e-09,7.472578e-10,1.000000e+00
4,1,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon at...,102,27,25,4,...,0.001328,-0.000561,0.003316,-0.001091,0.000047,-0.009183,-0.002713,8.960309e-01,1.016456e-01,2.323469e-03


In [45]:
del df_train['w2v_array']
df_train.columns.values

array(['author2', 'id', 'text', 'author', 'cleaned_text',
       'cleaned_text_string', 'length', 'num_words', 'num_unique_words',
       'num_punctuations', 'num_words_upper', 'num_words_title',
       'mean_word_len', 'num_stopwords', 'lexical_diversity', 'happi',
       'hand', 'mind', 'like', 'found', 'death', 'time', 'night', 'though',
       'hous', 'two', 'ever', 'much', 'year', 'never', 'come', 'even',
       'day', 'yet', 'know', 'life', 'natur', 'men', 'upon', 'well',
       'look', 'raymond', 'man', 'feel', 'shall', 'father', 'heart',
       'point', 'hope', 'great', 'dream', 'littl', 'howev', 'came',
       'strang', 'eye', 'street', 'room', 'door', 'friend', 'face',
       'heard', 'say', 'thing', 'made', 'must', 'place', 'word', 'appear',
       'thought', 'make', 'may', 'love', 'us', 'light', 'everi', 'die',
       'old', 'whose', 'might', 'pass', 'long', 'said', 'first', 'seem',
       'chang', 'mani', 'fear', 'one', 'thus', 'certain', 'still',
       'return', 'saw', '

In [46]:
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)

In [47]:
lemm = WordNetLemmatizer()
class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

In [48]:
# Storing the entire training text in a list
text = list(df_train.text.values)
# Calling our overwritten Count vectorizer
tf_vectorizer = LemmaCountVectorizer(max_df=0.95, 
                                     min_df=2,
                                     stop_words='english',
                                     decode_error='ignore')
tf = tf_vectorizer.fit_transform(text)

In [49]:
lda = LatentDirichletAllocation(n_components=13, max_iter=5,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0)

In [50]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=13, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [51]:
lda.transform(tf)

array([[ 0.00452489,  0.00452489,  0.00452489, ...,  0.00452499,
         0.06415156,  0.00452489],
       [ 0.01538462,  0.01538462,  0.01538467, ...,  0.01538462,
         0.01538462,  0.01538462],
       [ 0.00404858,  0.00404859,  0.89772543, ...,  0.00404861,
         0.00404865,  0.00404867],
       ..., 
       [ 0.00961538,  0.00961538,  0.00961556, ...,  0.0096154 ,
         0.00961538,  0.00961538],
       [ 0.01098901,  0.21741942,  0.01098901, ...,  0.01098901,
         0.01098901,  0.01098906],
       [ 0.00961538,  0.00961538,  0.13461525, ...,  0.00961563,
         0.00961538,  0.00961544]])

In [52]:
# Create data set
ds_train=df_train.values
X=ds_train[:, 6:]
Y=ds_train[:, 0]
seed=7
test_size=0.3
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X,Y, test_size=test_size, random_state=seed)

In [53]:
# fit model
xg_train=xgb.DMatrix(X_train, label=y_train)
xg_test=xgb.DMatrix(X_test, label=y_test)
xg_t=xgb.DMatrix(X, label=Y)
param={}
param['objective'] = 'multi:softmax'
param['eta'] = 0.2
param['max_depth'] = 2
param['silent'] = 1
param['num_class'] = 3
param['eval_metric']= "mlogloss"
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 240
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

[0]	train-mlogloss:0.921491	test-mlogloss:0.922556
[1]	train-mlogloss:0.798604	test-mlogloss:0.800504
[2]	train-mlogloss:0.708918	test-mlogloss:0.711609
[3]	train-mlogloss:0.641138	test-mlogloss:0.644234
[4]	train-mlogloss:0.589194	test-mlogloss:0.592531
[5]	train-mlogloss:0.54839	test-mlogloss:0.552153
[6]	train-mlogloss:0.516373	test-mlogloss:0.520772
[7]	train-mlogloss:0.490757	test-mlogloss:0.495507
[8]	train-mlogloss:0.469581	test-mlogloss:0.475061
[9]	train-mlogloss:0.452595	test-mlogloss:0.458734
[10]	train-mlogloss:0.438175	test-mlogloss:0.444398
[11]	train-mlogloss:0.426337	test-mlogloss:0.432876
[12]	train-mlogloss:0.416557	test-mlogloss:0.423428
[13]	train-mlogloss:0.407965	test-mlogloss:0.415124
[14]	train-mlogloss:0.400703	test-mlogloss:0.408265
[15]	train-mlogloss:0.394627	test-mlogloss:0.402537
[16]	train-mlogloss:0.389215	test-mlogloss:0.397656
[17]	train-mlogloss:0.384517	test-mlogloss:0.393553
[18]	train-mlogloss:0.380119	test-mlogloss:0.389497
[19]	train-mlogloss:0.3

[158]	train-mlogloss:0.265446	test-mlogloss:0.328779
[159]	train-mlogloss:0.265132	test-mlogloss:0.328772
[160]	train-mlogloss:0.264843	test-mlogloss:0.328665
[161]	train-mlogloss:0.264411	test-mlogloss:0.328436
[162]	train-mlogloss:0.264074	test-mlogloss:0.328456
[163]	train-mlogloss:0.263798	test-mlogloss:0.3284
[164]	train-mlogloss:0.263471	test-mlogloss:0.32849
[165]	train-mlogloss:0.263045	test-mlogloss:0.328378
[166]	train-mlogloss:0.262677	test-mlogloss:0.328241
[167]	train-mlogloss:0.262418	test-mlogloss:0.32808
[168]	train-mlogloss:0.262137	test-mlogloss:0.328082
[169]	train-mlogloss:0.261753	test-mlogloss:0.328064
[170]	train-mlogloss:0.261391	test-mlogloss:0.328045
[171]	train-mlogloss:0.261016	test-mlogloss:0.327993
[172]	train-mlogloss:0.260653	test-mlogloss:0.328087
[173]	train-mlogloss:0.260296	test-mlogloss:0.328146
[174]	train-mlogloss:0.259915	test-mlogloss:0.327958
[175]	train-mlogloss:0.259577	test-mlogloss:0.327984
[176]	train-mlogloss:0.259189	test-mlogloss:0.3278

In [54]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_test).reshape(y_test.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.921491	test-mlogloss:0.922556
[1]	train-mlogloss:0.798604	test-mlogloss:0.800504
[2]	train-mlogloss:0.708918	test-mlogloss:0.711609
[3]	train-mlogloss:0.641138	test-mlogloss:0.644234
[4]	train-mlogloss:0.589194	test-mlogloss:0.592531
[5]	train-mlogloss:0.54839	test-mlogloss:0.552153
[6]	train-mlogloss:0.516373	test-mlogloss:0.520772
[7]	train-mlogloss:0.490757	test-mlogloss:0.495507
[8]	train-mlogloss:0.469581	test-mlogloss:0.475061
[9]	train-mlogloss:0.452595	test-mlogloss:0.458734
[10]	train-mlogloss:0.438175	test-mlogloss:0.444398
[11]	train-mlogloss:0.426337	test-mlogloss:0.432876
[12]	train-mlogloss:0.416557	test-mlogloss:0.423428
[13]	train-mlogloss:0.407965	test-mlogloss:0.415124
[14]	train-mlogloss:0.400703	test-mlogloss:0.408265
[15]	train-mlogloss:0.394627	test-mlogloss:0.402537
[16]	train-mlogloss:0.389215	test-mlogloss:0.397656
[17]	train-mlogloss:0.384517	test-mlogloss:0.393553
[18]	train-mlogloss:0.380119	test-mlogloss:0.389497
[19]	train-mlogloss:0.3

[158]	train-mlogloss:0.265446	test-mlogloss:0.328779
[159]	train-mlogloss:0.265132	test-mlogloss:0.328772
[160]	train-mlogloss:0.264843	test-mlogloss:0.328665
[161]	train-mlogloss:0.264411	test-mlogloss:0.328436
[162]	train-mlogloss:0.264074	test-mlogloss:0.328456
[163]	train-mlogloss:0.263798	test-mlogloss:0.3284
[164]	train-mlogloss:0.263471	test-mlogloss:0.32849
[165]	train-mlogloss:0.263045	test-mlogloss:0.328378
[166]	train-mlogloss:0.262677	test-mlogloss:0.328241
[167]	train-mlogloss:0.262418	test-mlogloss:0.32808
[168]	train-mlogloss:0.262137	test-mlogloss:0.328082
[169]	train-mlogloss:0.261753	test-mlogloss:0.328064
[170]	train-mlogloss:0.261391	test-mlogloss:0.328045
[171]	train-mlogloss:0.261016	test-mlogloss:0.327993
[172]	train-mlogloss:0.260653	test-mlogloss:0.328087
[173]	train-mlogloss:0.260296	test-mlogloss:0.328146
[174]	train-mlogloss:0.259915	test-mlogloss:0.327958
[175]	train-mlogloss:0.259577	test-mlogloss:0.327984
[176]	train-mlogloss:0.259189	test-mlogloss:0.3278

In [55]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_t, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_t).reshape(Y.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != Y) / Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.921818	test-mlogloss:0.921863
[1]	train-mlogloss:0.79919	test-mlogloss:0.799002
[2]	train-mlogloss:0.709528	test-mlogloss:0.70911
[3]	train-mlogloss:0.641847	test-mlogloss:0.641432
[4]	train-mlogloss:0.58975	test-mlogloss:0.589307
[5]	train-mlogloss:0.549532	test-mlogloss:0.548942
[6]	train-mlogloss:0.51771	test-mlogloss:0.516905
[7]	train-mlogloss:0.492063	test-mlogloss:0.491249
[8]	train-mlogloss:0.470918	test-mlogloss:0.470304
[9]	train-mlogloss:0.453839	test-mlogloss:0.453268
[10]	train-mlogloss:0.439633	test-mlogloss:0.439011
[11]	train-mlogloss:0.42799	test-mlogloss:0.427375
[12]	train-mlogloss:0.417996	test-mlogloss:0.417243
[13]	train-mlogloss:0.409643	test-mlogloss:0.408651
[14]	train-mlogloss:0.402477	test-mlogloss:0.401332
[15]	train-mlogloss:0.396253	test-mlogloss:0.394994
[16]	train-mlogloss:0.39115	test-mlogloss:0.389764
[17]	train-mlogloss:0.386514	test-mlogloss:0.384979
[18]	train-mlogloss:0.382236	test-mlogloss:0.381012
[19]	train-mlogloss:0.378698

[158]	train-mlogloss:0.279266	test-mlogloss:0.282387
[159]	train-mlogloss:0.278998	test-mlogloss:0.282045
[160]	train-mlogloss:0.2786	test-mlogloss:0.281901
[161]	train-mlogloss:0.2782	test-mlogloss:0.281586
[162]	train-mlogloss:0.277894	test-mlogloss:0.281383
[163]	train-mlogloss:0.277572	test-mlogloss:0.281054
[164]	train-mlogloss:0.277269	test-mlogloss:0.280807
[165]	train-mlogloss:0.27697	test-mlogloss:0.28057
[166]	train-mlogloss:0.27671	test-mlogloss:0.280239
[167]	train-mlogloss:0.276429	test-mlogloss:0.279917
[168]	train-mlogloss:0.276223	test-mlogloss:0.279695
[169]	train-mlogloss:0.275988	test-mlogloss:0.279425
[170]	train-mlogloss:0.275705	test-mlogloss:0.279188
[171]	train-mlogloss:0.275471	test-mlogloss:0.278901
[172]	train-mlogloss:0.275226	test-mlogloss:0.278651
[173]	train-mlogloss:0.274851	test-mlogloss:0.278315
[174]	train-mlogloss:0.274599	test-mlogloss:0.278183
[175]	train-mlogloss:0.274324	test-mlogloss:0.277826
[176]	train-mlogloss:0.274049	test-mlogloss:0.277601


In [56]:
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,svd_word_43,svd_word_44,svd_word_45,svd_word_46,svd_word_47,svd_word_48,svd_word_49,nb_cvec_eap,nb_cvec_hpl,nb_cvec_mws
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-0.006320,0.019999,-0.006687,0.008188,-0.046846,-0.042702,0.000003,0.021018,0.000595,0.978387
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,0.003998,-0.000610,0.001042,-0.000802,-0.001725,0.004586,-0.006745,0.999985,0.000009,0.000006
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,0.007646,-0.004253,0.031135,0.000088,-0.001402,-0.012464,0.007291,0.217325,0.782527,0.000148


In [57]:
df_test['cleaned_text'] = df_test.text.apply(tokenize_stem)
df_test['cleaned_text_string'] = df_test.cleaned_text.apply(' '.join)
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,svd_word_45,svd_word_46,svd_word_47,svd_word_48,svd_word_49,nb_cvec_eap,nb_cvec_hpl,nb_cvec_mws,cleaned_text,cleaned_text_string
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-0.006687,0.008188,-0.046846,-0.042702,0.000003,0.021018,0.000595,0.978387,"[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,0.001042,-0.000802,-0.001725,0.004586,-0.006745,0.999985,0.000009,0.000006,"[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap gov...
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,0.031135,0.000088,-0.001402,-0.012464,0.007291,0.217325,0.782527,0.000148,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...


In [58]:
df_test['length']=df_test['cleaned_text_string'].apply(len)
df_test["num_words"] = df_test["text"].apply(lambda x: len(str(x).split()))
df_test["num_unique_words"] = df_test["text"].apply(lambda x: len(set(str(x).split())))
df_test["num_punctuations"] =df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test["num_words_upper"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_test["num_words_title"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_test["mean_word_len"] = df_test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test["num_stopwords"] = df_test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_test['lexical_diversity'] = df_test.text.apply(lexical_diversity)
df_test['w2v_array'] = df_test.cleaned_text.apply(sum_up_word2vec_array)
count_topwords(df_test)
create_w2v_columns(df_test) 
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,w2v_feature_90,w2v_feature_91,w2v_feature_92,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-1.759282,-0.576076,-0.398607,-3.074733,-2.463161,-2.642718,-4.210669,-1.410656,1.367103,4.731812
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,-2.596158,-1.332668,-1.035677,-7.674954,-4.623433,-7.287557,-9.342011,-4.080431,3.039795,10.461497
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,-0.008800,-0.943616,-0.712211,-5.862047,-1.784481,-6.222224,-5.740798,-3.634120,1.837138,6.516752


In [59]:
df_test['mws_index']=df_test['cleaned_text'].apply(ind_val_mws)/df_test['length']
df_test['eap_index']=df_test['cleaned_text'].apply(ind_val_eap)/df_test['length']
df_test['hpl_index']=df_test['cleaned_text'].apply(ind_val_hpl)/df_test['length']
df_test.head(n=3)
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99,mws_index,eap_index,hpl_index
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-3.074733,-2.463161,-2.642718,-4.210669,-1.410656,1.367103,4.731812,0.071473,0.036315,0.026541
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,-7.674954,-4.623433,-7.287557,-9.342011,-4.080431,3.039795,10.461497,0.035932,0.062884,0.039306
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,-5.862047,-1.784481,-6.222224,-5.740798,-3.634120,1.837138,6.516752,0.027486,0.055142,0.057723


In [60]:
del df_test['cleaned_text']
del df_test['cleaned_text_string']
del df_test['w2v_array']

In [61]:
cols=(df_train.columns.tolist())[6:]
[item for item in df_train.columns.tolist() if item not in df_test.columns.tolist()]

['author2', 'author', 'cleaned_text', 'cleaned_text_string']

In [62]:
df_test.head()

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,w2v_feature_93,w2v_feature_94,w2v_feature_95,w2v_feature_96,w2v_feature_97,w2v_feature_98,w2v_feature_99,mws_index,eap_index,hpl_index
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-3.074733,-2.463161,-2.642718,-4.210669,-1.410656,1.367103,4.731812,0.071473,0.036315,0.026541
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,-7.674954,-4.623433,-7.287557,-9.342011,-4.080431,3.039795,10.461497,0.035932,0.062884,0.039306
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,-5.862047,-1.784481,-6.222224,-5.740798,-3.634120,1.837138,6.516752,0.027486,0.055142,0.057723
3,id27757,While I was thinking how I should possibly man...,0.013408,-0.007515,-0.000154,-0.004020,-0.004521,0.002712,-0.004220,-0.002882,...,-7.262517,-3.616161,-7.041581,-7.981725,-3.806991,2.530908,9.024186,0.024369,0.065057,0.055736
4,id04081,I am not sure to what limit his knowledge may ...,0.012565,-0.003185,-0.000719,-0.001152,0.000930,-0.006110,0.001690,-0.002149,...,-1.902167,-1.957995,-1.297328,-2.738015,-0.694716,0.861141,3.303241,0.025615,0.070948,0.028437


In [63]:
df_test.columns

Index(['id', 'text', 'svd_word_0', 'svd_word_1', 'svd_word_2', 'svd_word_3',
       'svd_word_4', 'svd_word_5', 'svd_word_6', 'svd_word_7',
       ...
       'w2v_feature_93', 'w2v_feature_94', 'w2v_feature_95', 'w2v_feature_96',
       'w2v_feature_97', 'w2v_feature_98', 'w2v_feature_99', 'mws_index',
       'eap_index', 'hpl_index'],
      dtype='object', length=247)

In [64]:
ds_test=df_test[cols].values

In [65]:
x_t=ds_test[:, :]
y_t=df_test['id'].values
xg_t=xgb.DMatrix(x_t)
pred_prob = bstp.predict(xg_t).reshape(y_t.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)

In [66]:
print(pred_prob)

[[  3.38422842e-02   4.59687738e-03   9.61560845e-01]
 [  9.96320367e-01   2.27206061e-03   1.40760711e-03]
 [  6.60936609e-02   9.32789862e-01   1.11650804e-03]
 ..., 
 [  9.05344009e-01   5.72326817e-02   3.74233350e-02]
 [  2.18897983e-02   1.98120577e-03   9.76129055e-01]
 [  1.70164816e-02   9.82668400e-01   3.15122714e-04]]


In [67]:
export=pd.DataFrame(pred_prob)
export.insert(loc=0, column='id', value=y_t)
export.columns=['id','EAP', 'HPL', 'MWS']
export.head()

,id,EAP,HPL,MWS
0,id02310,0.033842,0.004597,0.961561
1,id24541,0.996320,0.002272,0.001408
2,id00134,0.066094,0.932790,0.001117
3,id27757,0.780370,0.216980,0.002650
4,id04081,0.982041,0.013768,0.004191


In [68]:
len(export)

8392

In [69]:
export[export['id']=='id23301']

,id,EAP,HPL,MWS
6106,id23301,0.337495,0.166845,0.49566


In [70]:
export.to_csv(path_or_buf="../data/export.csv", index=False)